In [ ]:
!pip install cryptography

In [ ]:
import zipfile
import os
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization

def generar_claves():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    public_key = private_key.public_key()
    return private_key, public_key

def guardar_llaves(private_key, public_key):
    with open("private_key.pem", "wb") as priv_file:
        priv_file.write(
            private_key.private_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.PKCS8,
                encryption_algorithm=serialization.NoEncryption()
            )
        )

    with open("public_key.pem", "wb") as pub_file:
        pub_file.write(
            public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            )
        )

def frimar(private_key, message): #usando la clave privada
    signature = private_key.sign(
        message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return signature

def verificar_firma(public_key, message, signature): #usando la clave publica

    try:
        public_key.verify(
            signature,
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except Exception as e:
        print(f"Error al verificar la firma: {e}")
        return False

def verificar_firma_zip(zip_file_path, public_key_path):
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zipf:
            message = zipf.read('message.txt')
            signature = zipf.read('signature.bin')

            # Cargar la clave pública desde el archivo
            with open(public_key_path, "rb") as pub_file:
                public_key = serialization.load_pem_public_key(pub_file.read())

            # Verificar la firma
            return verificar_firma(public_key, message, signature)
    except (zipfile.BadZipfile, FileNotFoundError, KeyError) as e:
        print(f"Error al procesar el archivo ZIP: {e}")
        return False

In [ ]:
# Generar claves
private_key, public_key = generate_keys()

# Guardar claves en archivos
guardar_llaves(private_key, public_key)

# Crear un archivo ZIP
with zipfile.ZipFile("signed_data.zip", "w") as zipf:
    zipf.writestr("message.txt", b"Hola, mundo!")
    zipf.writestr("signature.bin", frimar(private_key, b"Hola, mundo!"))

# Verificar la firma del archivo ZIP
is_valid = verificar_firma_zip("signed_data.zip", "public_key.pem")
if is_valid:
    print("La firma del archivo ZIP es válida.")
else:
    print("La firma del archivo ZIP es inválida.")

La firma del archivo ZIP es válida.
